## 特徵提取

In [6]:
Ball_x = []
Ball_y = []
Ball_speed_x = []
Ball_speed_y = []
Direction = []
Platform = []
Command = []

In [7]:
from tqdm.notebook import tqdm 
import pickle
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
path = 'C:\\Users\\user\\Desktop\\學校課程\\資工系\\機器學習_遊戲\\HW1\\code\\game\\MLGame-master\\MLGame-master\\games\\arkanoid\\log\\'
with tqdm (total = len(os.listdir(path)),desc = 'feature extract') as pbar:
    for pick in os.listdir(path):
        file_path = path + pick

        with open(file_path, 'rb') as f:
            data = pickle.load(f)
            scene_info = data['ml']['scene_info']
            command = data['ml']['command']


            for i, s in enumerate(scene_info[1:-2]):
                Ball_x.append(s['ball'][0])
                Ball_y.append(s['ball'][1])
                Platform.append(s['platform'][0])
                Ball_speed_x.append(scene_info[i+2]["ball"][0] - scene_info[i+1]["ball"][0])
                Ball_speed_y.append(scene_info[i+2]["ball"][1] - scene_info[i+1]["ball"][1])
                if Ball_speed_x[-1] > 0:
                    if Ball_speed_y[-1] > 0:
                        # 右下
                        Direction.append(0)
                    else:
                        # 右上
                        Direction.append(1)
                else:
                    if Ball_speed_y[-1] > 0:
                        # 左下
                        Direction.append(2)
                    else:
                        # 左上
                        Direction.append(3)

            for c in command[1:-2]:
                if c == "NONE":
                    Command.append(0)
                elif c == "MOVE_LEFT":
                    Command.append(1)
                elif c == "MOVE_RIGHT":
                    Command.append(2)
        pbar.update(1)

feature extract:   0%|          | 0/29 [00:00<?, ?it/s]

In [19]:
import numpy as np

numpy_data = np.array([Ball_x, Ball_y, Ball_speed_x, Ball_speed_y, Direction, Platform])
X = np.transpose(numpy_data) 
y = Command

## 特徵挑選

In [23]:
from sklearn import feature_selection as FS

In [45]:
FS.f_classif(np.transpose(np.array([Ball_speed_x, Ball_speed_y])),
             Direction)[1] < 0.05

array([ True,  True])

In [24]:
fv, pv = FS.f_classif(X, y)

In [35]:
print(fv)

[  151.82867   1108.7435   13190.266    17088.814    14960.611
    34.284622]


In [36]:
pv

array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 1.3204936e-15], dtype=float32)

In [33]:
pv < 0.05

array([ True,  True,  True,  True,  True,  True])

## 模型訓練

In [6]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('my_model_8.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    1.6s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    1.7s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    1.7s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.8s finished


In [7]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 1}
[[4400    0    0]
 [   0 4991    2]
 [   0    5 5374]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4400
           1       1.00      1.00      1.00      4993
           2       1.00      1.00      1.00      5379

    accuracy                           1.00     14772
   macro avg       1.00      1.00      1.00     14772
weighted avg       1.00      1.00      1.00     14772

